<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Red%20convolucional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import tensorflow.keras as keras
import sklearn as sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import MaxPooling1D
from keras.layers import Conv1D
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras import layers
from google.colab import files
from sklearn.metrics import roc_auc_score
from sklearn import metrics
!pip install talos
import talos

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
# CARGAMOS LOS CSV
url_train = 'https://raw.githubusercontent.com/JulioCastillo1/TP1-Datos/master/train.csv'
url_test = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test.csv'

train_df = pd.read_csv(url_train)
test_df = pd.read_csv(url_test)
train_df = train_df.drop(['keyword', 'location','id'], axis = 1)
train_df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
train_df.shape

(7613, 2)

In [4]:
x_train, x_validation, y_train, y_validation = train_test_split(train_df['text'],train_df['target'],random_state=7,test_size = 0.20)
x_train = train_df['text']
y_train = train_df['target']

In [5]:
def modelardo(x_train,y_train,x_test,y_test,params):
  vectorizer = CountVectorizer()
  vectorizer.fit(x_train)

  x_train = vectorizer.transform(x_train)
  x_test = vectorizer.transform(x_test)
  input_dim = x_train.shape[1]
  model = Sequential()
  model.add(layers.Dense(params['first neuron units'],input_dim=input_dim,activation=params['first neuron activation']))
  model.add(layers.Dense(1,activation = 'sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  out = model.fit(x_train,y_train,epochs = params['epochs'],verbose = 0,validation_data = (x_test,y_test),batch_size=params['batch size'])
  return out,model


In [6]:
x_train.shape
#t = talos.Scan(x_train,y_train,params=p,model=modelardo,experiment_name='diabetes')
#Este proceso tarda 1,5hs aprox en colab. Y tiene muy poquitos hiperparámetros para tunear

(7613,)

In [7]:
#experiment_data = t.data
#experiment_data.head()

In [8]:
#experiment_data['val_accuracy'].idxmax()

In [9]:
#experiment_data.iloc[205]

In [10]:
#vectorizer = CountVectorizer()
#vectorizer.fit(train_df['text'])

#data_train = vectorizer.fit_transform(train_df['text']).toarray()


In [11]:
# Función con el modelo para pasar por parámetro a talos.

def modelo(x_train,y_train,x_test,y_test,params):
  tokenizer = Tokenizer(num_words=2000)
  tokenizer.fit_on_texts(x_train)

  train_sequences = tokenizer.texts_to_sequences(x_train)
  validation_sequences = tokenizer.texts_to_sequences(x_validation)
  train_data = pad_sequences(train_sequences, maxlen=100)
  validation_data = pad_sequences(validation_sequences, maxlen=100)


  #input_dim = x_train.shape[1]

  best_model = Sequential()
  best_model.add(layers.Embedding(2000, 100, input_length=100))
  best_model.add(Conv1D(128, 5, activation = params['activation']))
  best_model.add(layers.GlobalAveragePooling1D())
  #best_model.add(Conv1D(512, 5, activation='relu'))
  #best_model.add(layers.GlobalAveragePooling1D())
  #best_model.add(Conv1D(512, 5, activation='relu'))
  #best_model.add(layers.GlobalAveragePooling1D())
  #best_model.add(MaxPooling1D(5))
  #best_model.add(layers.Dropout(0.25))
  #best_model.add(Flatten())
  #best_model.add(LSTM(64))
  best_model.add(layers.Dense(128, activation = params['activation']))
  #best_model.add(layers.Dense(128 ,activation='relu'))
  best_model.add(layers.Dense(units = 1, activation='sigmoid'))
  #best_model.add(layers.Dense(1,activation = 'sigmoid'))
  best_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  out = best_model.fit(train_data,y_train,epochs = params['epochs'],verbose = 1,validation_data = (validation_data,y_validation),batch_size=params['batch size'])
  return out, best_model

#loss, accuracy = best_model.evaluate(train_data, y_train, verbose=False)
#print("Training Accuracy: {:.4f}".format(accuracy))
#loss, accuracy = best_model.evaluate(validation_data, y_validation, verbose=False)
#print("Testing Accuracy:  {:.4f}".format(accuracy))

p = {
    'neuron_units': [20,32,64,128,256,512],
    'activation' : ['relu','elu','softmax','tanh','exponential'],
    'epochs' : [1,2,3,4,5,10],
    'batch size': [5,100,5],
}

#t = talos.Scan(x_train, y_train, params = p, model = modelo, experiment_name = 'Convo hyperparam tuning')

In [12]:
# Divide el set en train y validation, y entrena con el modelo tomando los hiperparámetros sugeridos

x_train, x_test, y_train, y_test = train_test_split(train_df['text'],train_df['target'],random_state=7,test_size = 0.20)
  
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(x_train)

train_sequences = tokenizer.texts_to_sequences(x_train)
validation_sequences = tokenizer.texts_to_sequences(x_validation)
train_data = pad_sequences(train_sequences, maxlen=100)
validation_data = pad_sequences(validation_sequences, maxlen=100)


#input_dim = x_train.shape[1]

best_model = Sequential()
best_model.add(layers.Embedding(2000, 100, input_length=100))
best_model.add(Conv1D(128, 5, activation='elu'))
best_model.add(layers.GlobalAveragePooling1D())
#best_model.add(Conv1D(512, 5, activation='relu'))
#best_model.add(layers.GlobalAveragePooling1D())
#best_model.add(Conv1D(512, 5, activation='relu'))
#best_model.add(layers.GlobalAveragePooling1D())
#best_model.add(MaxPooling1D(5))
#best_model.add(layers.Dropout(0.25))
#best_model.add(Flatten())
#best_model.add(LSTM(64))
best_model.add(layers.Dense(128 ,activation='elu'))
#best_model.add(layers.Dense(128 ,activation='relu'))
best_model.add(layers.Dense(1,activation='sigmoid'))
#best_model.add(layers.Dense(1,activation = 'sigmoid'))
best_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
out = best_model.fit(train_data,y_train,epochs=5,verbose = 1,validation_data = (validation_data,y_validation),batch_size=50)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 6090 samples, validate on 1523 samples
Epoch 1/5
6090/6090 [==============================] - 7s 1ms/step - loss: 0.6226 - accuracy: 0.6378 - val_loss: 0.4953 - val_accuracy: 0.7722
Epoch 2/5
6090/6090 [==============================] - 7s 1ms/step - loss: 0.4379 - accuracy: 0.8064 - val_loss: 0.4580 - val_accuracy: 0.8017
Epoch 3/5
6090/6090 [==============================] - 7s 1ms/step - loss: 0.3820 - accuracy: 0.8374 - val_loss: 0.4596 - val_accuracy: 0.8076
Epoch 4/5
6090/6090 [==============================] - 7s 1ms/step - loss: 0.3462 - accuracy: 0.8499 - val_loss: 0.4725 - val_accuracy: 0.8024
Epoch 5/5
6090/6090 [==============================] - 7s 1ms/step - loss: 0.3259 - accuracy: 0.8622 - val_loss: 0.4775 - val_accuracy: 0.8043


In [13]:
prediction=best_model.predict_classes(validation_data, batch_size=50)
score = roc_auc_score(y_validation, prediction)
score

0.7941268320753988

In [14]:
accuracy=metrics.accuracy_score(y_validation, prediction)
accuracy

0.804333552199606

In [15]:
#Reentrena con el set completo y realiza predicción.

#set_sequences = tokenizer.texts_to_sequences(train_df['text'])
#set_data = pad_sequences(set_sequences, maxlen=100)

#out = best_model.fit(set_data,train_df['target'],epochs = 5,verbose = 1,batch_size=62)

#out = best_model.fit(data_train,train_df['target'],epochs = 5,verbose = 1,batch_size=62)
#data_test = vectorizer.transform(test_df['text']).toarray()
test_prediction = tokenizer.texts_to_sequences(test_df['text'])
prediction_data = pad_sequences(test_prediction, maxlen=100)

prediction2=best_model.predict_classes(prediction_data, batch_size=50)
prediction2

array([[0],
       [1],
       [1],
       ...,
       [1],
       [1],
       [0]], dtype=int32)

In [16]:
# Genera archivo submit.

test_df['target'] = prediction2
export = pd.DataFrame()
export['id'] = test_df['id']
export['target'] = test_df['target']
export = export.set_index('id')
export.to_csv('submit.csv')
#files.download('submit.csv')